In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [3]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    # Create a usable list of points from the contours
    sub_contours = []
    shrinked_list = []

    for contour in contours:
        sub_contour_points = []
        for contour_points in contour:
            sub_contour_points.append(contour_points[0])
        sub_contours.append(sub_contour_points)

        # Shrink the list of contours 
        shrinked_sub_list = []

        for i in range(0, len(sub_contour_points)):
            if i % 2 == 0: shrinked_sub_list.append(sub_contour_points[i])
        shrinked_list.append(shrinked_sub_list)

    return np.array(shrinked_list)

In [4]:
def get_class(filename):
    num = filename.split('_')[1].split('.')[0]
    
    if num == "1":
        return "one"
    elif num == "2":
        return "two"
    elif num == "3":
        return "three"
    elif num == "4":
        return "four"
    elif num == "5":
        return "five"
    elif num == "6":
        return "six"

In [5]:
class Mask(object):
    def __init__(self, file):
        self.name = file.split('/')[2]
        self.classification = get_class(self.name)
        self.contours = find_contours(cv2.imread(file))
        

class Image(object):
    def __init__(self, file):
        self.image = cv2.imread(file)
        self.name = file.split('/')[2]
        self.masks = self.get_masks("./masks/" + file.split('/')[2].split('.')[0] + "*.png")
        
    def get_masks(self, path):
        masks = []
        for file in glob.glob(path):
            masks.append(Mask(file))
            
        return masks
        

In [6]:
# Save the points to a json file
images = []

for file in glob.glob("./images/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    regions = []
    for mask in image.masks:
        region = {}
        region['name'] = mask.name
        region['type'] = "polyline"
        region['class'] = mask.classification
        x_points = []
        y_points = []
        for points in mask.contours:
            points = np.array(points)
            x_points.append([points.transpose()[0].tolist()] if len(points) > 0 else [])
            y_points.append([points.transpose()[1].tolist()] if len(points) > 0 else [])
        region['all_points_x'] = x_points
        region['all_points_y'] = y_points
        
        regions.append(region)
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['regions'] = regions
    
    data['image_' + image.name] = image_information
    
with open('annotations.json', 'w') as json_file:
    json.dump(data, json_file)

In [7]:
#For testing
import random

annotations = json.load(open(os.path.join("annotations.json")))
annotations = list(annotations.values())

for annotation in annotations:
    fin_img = cv2.imread("./images/" + annotation['filename'])

    for region in annotation['regions']:    
        x_points = region['all_points_x']
        y_points = region['all_points_y']
        
        color = random.choice([[0,0,255], [255, 0, 0], [0, 255, 0], [255, 255, 0], [0, 255, 255], [255, 0, 255]])
        
        for x, y in zip(x_points, y_points):
            points = np.array([x, y]).transpose()

            cv2.polylines(fin_img, [points], True, color, 2)

    # Show
    cv2.imshow(annotation['filename'], fin_img)

cv2.waitKey(0)
cv2.destroyAllWindows()
    